
<a href="https://colab.research.google.com/github/takzen/pytorch-black-belt/blob/main/22_Buffers_vs_Parameters.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [ ]:
# --------------------------------------------------------------
# ☁️ COLAB SETUP (Automatyczna instalacja środowiska)
# --------------------------------------------------------------
import sys
import os

# Sprawdzamy, czy jesteśmy w Google Colab
if 'google.colab' in sys.modules:
    print('☁️ Wykryto środowisko Google Colab. Konfiguruję...')

    # 1. Pobieramy plik requirements.txt bezpośrednio z repozytorium
    !wget -q https://raw.githubusercontent.com/takzen/ai-engineering-handbook/main/requirements.txt -O requirements.txt

    # 2. Instalujemy biblioteki
    print('⏳ Instaluję zależności (to może chwilę potrwać)...')
    !pip install -q -r requirements.txt

    print('✅ Gotowe! Środowisko jest zgodne z repozytorium.')
else:
    print('💻 Wykryto środowisko lokalne. Zakładam, że masz już uv/venv.')


# 🥋 Lekcja 22: Buffers vs Parameters (Stan nietrenowalny)

W PyTorch `nn.Module` ma dwa główne magazyny:
1.  **Parameters:** Tensory, które mają gradient i są aktualizowane przez optymalizator (Wagi, Biasy).
2.  **Buffers:** Tensory, które są częścią stanu modelu (zapisują się w `state_dict`, przenoszą się na GPU), ale **nie są trenowane** przez gradient.

**Typowy błąd:**
Przypisanie tensora jako zwykłego atrybutu (`self.t = torch.randn(5)`).
*   Skutek 1: Nie przeniesie się na GPU przy `model.cuda()`.
*   Skutek 2: Nie zapisze się przy `torch.save()`.

W tej lekcji naprawimy ten błąd.

In [1]:
import torch
import torch.nn as nn

# Definiujemy moduł z 3 rodzajami zmiennych
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        # 1. Parameter (Standard)
        # To będzie trenowane.
        self.weight = nn.Parameter(torch.randn(3, 3))
        
        # 2. Zwykły atrybut (BŁĄD!)
        # To jest "niewidzialne" dla silnika PyTorch.
        self.mistake_tensor = torch.randn(3, 3)
        
        # 3. Buffer (POPRAWNIE)
        # To jest stan nietrenowalny (np. średnia w BatchNorm).
        # Musimy użyć register_buffer(nazwa, tensor)
        self.register_buffer('running_mean', torch.randn(3, 3))

model = MyModel()

print("Model zainicjowany.")

Model zainicjowany.


## Test 1: Przenoszenie na GPU (`.to(device)`)

To jest najczęstsza przyczyna błędu `RuntimeError: Expected all tensors to be on the same device`.
Użytkownik pisze `model.to('cuda')`, ale jego pomocniczy tensor zostaje na CPU.

In [2]:
# Sprawdźmy, czy masz GPU (jeśli nie, test pokażemy na idei)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Przenoszę model na: {device}")

model.to(device)

print("\n--- GDZIE SĄ DANE? ---")
print(f"Weight (Param):       {model.weight.device}")
print(f"Running Mean (Buffer): {model.running_mean.device}")
print(f"Mistake (Zwykły):      {model.mistake_tensor.device}")

if str(device) != 'cpu' and str(model.mistake_tensor.device) == 'cpu':
    print("\n❌ BŁĄD! 'mistake_tensor' został na CPU, mimo że model jest na GPU.")
    print("W obliczeniach to spowoduje awarię.")

Przenoszę model na: cuda

--- GDZIE SĄ DANE? ---
Weight (Param):       cuda:0
Running Mean (Buffer): cuda:0
Mistake (Zwykły):      cpu

❌ BŁĄD! 'mistake_tensor' został na CPU, mimo że model jest na GPU.
W obliczeniach to spowoduje awarię.


## Test 2: Zapisywanie modelu (`state_dict`)

Czy nasze dane przetrwają zapis do pliku?

In [3]:
state = model.state_dict()

print("--- CO JEST W PLIKU ZAPISU? ---")
print(state.keys())

# Weryfikacja
if 'running_mean' in state:
    print("✅ Buffer: JEST ZAPISANY.")
else:
    print("❌ Buffer: BRAK.")

if 'mistake_tensor' in state:
    print("✅ Mistake: JEST ZAPISANY.")
else:
    print("❌ Mistake: BRAK (Zgubiliśmy dane!).")

--- CO JEST W PLIKU ZAPISU? ---
odict_keys(['weight', 'running_mean'])
✅ Buffer: JEST ZAPISANY.
❌ Mistake: BRAK (Zgubiliśmy dane!).


## Test 3: Optymalizator

Czy optymalizator (np. Adam) spróbuje zmienić nasze wartości?
Buforów nie powinien dotykać.

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

print("--- CO WIDZI OPTYMALIZATOR? ---")
# Optymalizator iteruje po parametrach
param_names = [n for n, p in model.named_parameters()]

print(f"Trenowane parametry: {param_names}")

if 'running_mean' not in param_names:
    print("✅ Buffer jest bezpieczny (nie będzie modyfikowany przez SGD).")

--- CO WIDZI OPTYMALIZATOR? ---
Trenowane parametry: ['weight']
✅ Buffer jest bezpieczny (nie będzie modyfikowany przez SGD).


## 🥋 Black Belt Summary

**Kiedy używać `register_buffer`?**

1.  **BatchNorm / LayerNorm:** Przechowywanie średniej i wariancji, które są aktualizowane matematycznie, a nie gradientem.
2.  **Positional Encodings (Transformer):** Macierz sinusów jest stała. Musi być na GPU, musi być zapisana z modelem, ale nie może się zmieniać.
3.  **Maski:** Maski przyczynowe (Causal Mask) w GPT.
4.  **Liczniki:** Np. `self.register_buffer('steps', torch.tensor(0))`, jeśli chcesz, żeby model wiedział, ile kroków już trenował (i żeby to się zapisało w checkpoincie).

Jeśli napiszesz `self.cos_table = ...` w Transformerze – popełniasz błąd. Użyj `register_buffer`.